In [1]:
import urllib.request
from bs4 import BeautifulSoup
import bs4
import csv
import re

In [2]:
def downloaden(link: str) -> (str):
    user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
    req = urllib.request.Request(link, headers={'User-Agent': user_agent})
    with urllib.request.urlopen(req) as resp:
        html = resp.read()
    return html

In [3]:
def getLinksFromPivot(root_link):
    html = downloaden(root_link)
    output = {}
    lines = str(html.decode('windows-1251')).split('\n')
    for line in lines:
        if len(re.findall('href=', line)) > 3:
            soup = BeautifulSoup(line, 'html.parser')
            links = soup.find_all('a')
            for link in links:
                if re.search('>(.+?)<', str(link)):
                    name = re.search('>(.+?)<', str(link)).group(1)
                    href = link.get('href')
                    output[name] = href
            break
    return output

In [ ]:
getLinksFromPivot('http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100028713304&vrn=100100028713299&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100028713304&type=233')

In [4]:
def bridgeToLocal(link, string='комиссии субъекта'):
    html = downloaden(link)
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all('a')
    output = ''
    for link in links:
        text = link.text
        if string in text.strip():
            output = link.get('href')
    return output

In [5]:
def mixItUp(details_dict, positions):
    #positions.extend([20, 21, 22, 23, 24, 25, 26])
    output = {}
    uiks = details_dict[0]
    for number, uik in enumerate(uiks):
        wanted_list = []
        for i in positions:
            value = details_dict[i][number]
            if '\n' in value:
                value = value.split('\n')[0]
            wanted_list.append(value)
        output[uik] = wanted_list
    return output

In [6]:
def getTable(link, positions):
    html = downloaden(link)
    soup = BeautifulSoup(html, 'html.parser')
    tables = soup.find_all('table')
    #meta = tables[-2].find_all('tr')
    details = tables[-1].find_all('tr')
    #meta_cells = [row.find_all('td') for row in meta]
    details_cells = [row.find_all('td') for row in details]

    #meta_f = {}
#for row_id, row in enumerate(meta_cells):
    #fertig = [cell.text.strip() for cell in row]
    #meta_f[row_id] = fertig[1]
    details_f = {}

    for row_id, row in enumerate(details_cells):
        fertig = [cell.text.strip() for cell in row]
        details_f[row_id] = fertig
    output = mixItUp(details_f, positions)
    return output

In [19]:
def updateTable(region, wb):
    with open('duma2013.csv', 'a') as f:
        tiks = wb[region].keys()
        for tik in tiks:
            uiks = wb[region][tik].keys()
            for uik in wb[region][tik]:
                uik_n = 'УИК №' + uik[5:]
                row = [region, tik, uik_n]
                row.extend(wb[region][tik][uik])
                try:
                    f.write(';'.join(row))
                except UnicodeEncodeError:
                    print(row)
                f.write('\n')

In [8]:
def getMeta(link, positions):
    html = downloaden(link)
    soup = BeautifulSoup(html, 'html.parser')
    tables = soup.find_all('table')
    meta = tables[-2].find_all('tr')
    meta_cells = [row.find_all('td') for row in meta]
    
    meta_f = []
    for row_id, row in enumerate(meta_cells):
        fertig = [cell.text.strip() for cell in row]
        if row_id < 20:
            meta_f.append(fertig[1])
        else:
            meta_f.append(fertig[1].split('партия ')[1].strip('"'))
    return meta_f

In [14]:
def main(positions=[1,7,8,9,10], counter=0):
    positions.extend([20, 21, 22, 23, 24, 25, 26])
    root_pivot = 'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100028713304&vrn=100100028713299&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100028713304&type=233'
    oik_links = getLinksFromPivot(root_pivot)
    print(len(oik_links))
    oik_tik = {}
    
    if counter == 0:
        fill = open('duma2013.csv', 'w')
        starter = 0
    else:
        starter = 1
    
    for oik in list(oik_links.keys())[counter:]:
        oik_tik[oik] = {}
        o_href = oik_links[oik]
        tik_links = getLinksFromPivot(o_href)
        if tik_links:
            for tik in tik_links:
                t_href = tik_links[tik]
                table_link = bridgeToLocal(t_href)
                oik_tik[oik][tik] = getTable(table_link, positions)
        else:
            tik = '-'
            table_link = bridgeToLocal(o_href)
            oik_tik[oik][tik] = getTable(table_link, positions)
        
        if starter == 0:
                    meta = getMeta(table_link, positions)
                    headers = ["ОИК", "ТИК", "УИК"]
                    headers.extend(meta)
                    fill.write(';'.join(headers))
                    fill.write('\n')
                    fill.close()
                    starter = 1
        
        updateTable(oik, oik_tik)
        counter += 1
        print('%s\t%s' % (str(counter), oik))

In [20]:
main(counter=134)

135
135	Территория за пределами РФ
